<a href="https://colab.research.google.com/github/jlschwartz3/bio108codingtutorial/blob/main/Data_Science_Coding_Tutorial_(Draft).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rain, rain, go away? Or rain, rain, come? Precipitation and global agriculture

Across the world, scientists are working to predict the effects of climate change on spatial and temporal levels. Global climate models, or GCMs, attempt to mathematically represent the complex processes of the Earth's climate system, including land surface, oceans, and the atmosphere. In the face of accelerating climate change, these models can be used to predict future climate data.

Globally, cycles of rainfall dictate when to sow and harvest crops. Expected precipitation values for given months are essential to planning tilling, purchasing seeds, and determining if certain crops can continue to thrive in certain regions.

Using projected climate data, we will examine how precipitation seasonality varies across regions and how that could impact agricultural patterns in the next 2 decades.

The data come from WorldClim's future climate data projections for 2021-2040. It comes from the EC-Earth3-Veg GCM, a model that focuses on changes in vegetation types and distribution in response to climate change, and uses the Shared Socio-economic Pathway (SSP) 126, which refers to a low-emissions projection of future radiative forcing. The data is at the 10 minute spatial resolution, the most coarse of WorldClim's data.

In [ ]:
! pip install rasterio fiona #install rasterio library, which is not pre-loaded into Colab

```
import rasterio #install all necessary packages for working with raster data
import rasterio.plot
import matplotlib.pyplot as plt
```

In [ ]:
import rasterio #install all necessary packages for working with raster data
import rasterio.plot
import matplotlib.pyplot as plt

For today's tutorial, we will start with working with raster data, where pixels are organized into larger grids to display geographic information.

```
raster_path = (
    "https://github.com/jlschwartz3/bio108tutorial/blob/main/wc2.1_10m_prec_EC-Earth3-Veg_ssp126_2021-2040.tif"
) #read in the raster data from .tif file, which was uploaded into Drive for ease of download
src = rasterio.open(raster_path)
print(src)
```

```
src.meta #examine metadata. Important: shows coordinate reference system and number of bands present, 12.
```

```
rasterio.plot.show(src) #plot the data

fig, ax = plt.subplots(figsize=(8, 8)) #plot again, but introduce a new colormap to better visualize data
rasterio.plot.show(src, cmap="viridis_r", ax=ax)
plt.show()
```

Plot the data separated out by band (month)
```
band_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"] #name all of the 12 bands present in the
#dataset, which correspond to the 12 months of the year

fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(10, 18)) #create 6 rows and 2 columns for clear visual output
axes = axes.flatten()  #flatten the 2D array of axes to 1D for easy iteration

for band in range(1, src.count + 1): #include the +1 so that the final band is still included; all bands will be plotted in separate plots
    data = src.read(band)
    ax = axes[band - 1]
    im = ax.imshow(data, cmap="viridis_r", vmin=0, vmax=1000) #use viridis_r colormap to show contrast between yellow (low precip) and blue/green (higher precip)
    ax.set_title(f"Band {band_names[band - 1]}")
    fig.colorbar(im, ax=ax, label="Precipitation (mm)", shrink=0.25) #add in colorbar scale to help interpret precipitation values

plt.tight_layout()
plt.show()
```

### Geographic clipping

For the next analysis, let's zoom in on a specific country to take a closer look at its unique precipitation patterns: Brazil.

As of 2022, Brazil was the world's second-largest exporter of agricultural products and provides food security to much of Latin America. Its soybeans, corn, beef, and coffee are are also staples for China, which sources nearly a quarter of its imports from Brazil. Crops coming from Brazil feed approximately 10% of the the world.

From a climate perspective, Brazil plays a crucial role in the regulation of global patterns through the influence of the Amazon rainforest, much of which lies within Brazil's boundaries. Evapotranspiration from the rainforest results in increased water vapor in the atmosphere, which condenses and falls as precipitation near and far, depending on wind patterns. The Amazon rainforest is responsible for around 50% of all rainfall in the region, supplying moisture to farmers across the basin.

How much does Brazil's rainfall fluctuate throughout the year? To optimize agriculture, it's important for farmers to have a strong understanding of precipitation patterns for each month.

In [ ]:
!npm install --save brazilian-boundaries #brazilian-boundaries file sourced from GitHub

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙npm warn deprecated core-js@2.6.12: core-js@<3.23.3 is no longer maintained and not recommended for usage due to the number of issues. Because of the V8 engine whims, feature detection in old core-js versions could cause a slowdown up to 100x even if nothing is polyfilled. Some versions have web compatibility issues. Please, upgrade your dependencies to the actual version of core-js.
⠙⠹⠸⠼⠴
added 5 packages in 4s
⠴

Here, we're going to read in a GeoJSON file, which is a type of vector file, or spatial data shown through lines, points, and polygons. In this case, it's a polygon representing the boundaries of Brazil.

```
import fiona
import rasterio.mask
import geopandas as gpd

geojson_path = "https://github.com/jlschwartz3/bio108tutorial/blob/main/brazil_Brazil_Country_Boundary.geojson"
bounds = gpd.read_file(geojson_path) #import geoJSON vector file showing Brazil's national boundaries

fig, ax = plt.subplots()
rasterio.plot.show(src, ax=ax)
bounds.plot(ax=ax, edgecolor="red", facecolor="none") #plot the original plot, with the Brazil boundary vector overlaid
```



```
import matplotlib.pyplot as plt
import numpy as np

with fiona.open(geojson_path, "r") as f:
    shapes = [feature["geometry"] for feature in f]

out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True) #clips raster file to just show the desired feature (Brazil)

global_vmin = np.nanmin(out_image)  #minimum value across all bands
global_vmax = np.nanmax(out_image)  #maximum value across all bands

num_bands = out_image.shape[0]

fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(10, 18))
axes = axes.flatten()

for i in range(num_bands):
    band_data = out_image[i]

    ax = axes[i]
    im = ax.imshow(band_data, cmap="viridis_r", vmin=global_vmin, vmax=global_vmax)
    ax.set_title(f"Band {i + 1}") #bands correspond to months of the year
    cbar = plt.colorbar(im, ax=ax, shrink=0.6)
    cbar.set_label("Precipitation (mm)", fontsize=10)


plt.tight_layout()
plt.show()
```

The clipped raster files show that precipitation is generally heaviest in the December-April months, but that it varies by region. Certain parts in the northwestern area, which is densely forested, are rainiest May-July, whereas some heavily agricultural zones in the south-central region have a shorter "wet season," with heaviest precipitation concentrated in just December and January.

# Climatic variation and crop yield



So how does this connect to conservation of agriculture?

In a paper published in Nature Communications in 2015, [Ray et al.](https://doi.org/10.1038/ncomms6989) examined the influence of climatic variation on fluctuation of yields for major global crops.

They found that climate variability accounts for approximately one-third (32–39%) of the observed annual yield variability for maize, rice, wheat, and soybean globally. This is huge!

Furthermore, the impact of climate factors varies by region. In some places, temperature is more influential, whereas precipitation is more important in others.

For our analysis, we'll focus on precipitation.



Citation:
Ray, D., Gerber, J., MacDonald, G. et al. Climate variation explains a third of global crop yield variability. Nat Commun 6, 5989 (2015). https://doi.org/10.1038/ncomms6989




### **Our objectives**


1. Visualize the global patterns of crop yield variability based on the time series data using Leafmap.
2. Analyze whether total crop yield variability explained due to climate variability is higher in wetter or drier regions.
3. Ultimately, narrow this analysis to Brazil, a key producer of maize, to compare with earlier precipitation analysis.

To execute this analysis, we will use the same dataset that Ray et al. 2015 used to produce their data. It's available in EarthStat's collection of geographic datasets on land use and agriculture.

We will use the Leafmap package to create an interactive visualization.

In [ ]:
! pip install -U leafmap #install leafmap, fiona, and rasterio packages
! pip install fiona
! pip install folium matplotlib mapclassify
! pip install localtileserver
! pip install rasterio

Leafmaps show changes in data using color scales. Because Ray et al.'s data is categorical, the colors can't be represented as a continuous scale, like in the previous visualizations. Instead, they identified 8 different categories, and all of the data will fall into one of those 8 "buckets" and be represented by a very defined color.

Their categories were: No Data/Ocean, No Effect, 0–15%, >15–30%, >30–45%, >45–60%, >60–75%, >75%, with the percentages representing the percent of crop yield variability that was explained by variation in climate.

```
import leafmap #import packages
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

#define category values and labels (including 0 as no data/ocean)
categories = [0, 1, 2, 3, 4, 5, 6, 7]
labels = [
    "No Data / Ocean", "No Effect", "0–15%", ">15–30%", ">30–45%", ">45–60%", ">60–75%", ">75%"
]

#define colors and set gray as color for no data/ocean areas
spectral_cmap = plt.get_cmap('Spectral')
color_values = [mcolors.to_rgba('lightgray')] + [spectral_cmap(i / 6) for i in range(7)]
cmap = mcolors.ListedColormap(color_values)
vmin = 0
vmax = 7

#path to the .tif file on total crop yield variability explained due to climate variability for maize
tif = '/content/drive/MyDrive/crop yields/explanatorycat_maize.tif'

#create the map
m = leafmap.Map(center=(33, 12), zoom=3, height="590px")

#add in the colorbar
m.add_raster(
    tif,
    layer_name="Maize Raster", #add the raster for format with only 1 band and no band descriptions
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    nodata=0,  # optional if 0 is no data
    opacity=1,
)

#add in text to legend
legend_dict = dict(zip(labels, [mcolors.to_hex(c) for c in color_values]))
m.add_legend(title="Maize Variability Explained by Climate", legend_dict=legend_dict)

m
```

The resulting Leafmap shows that in darker blue and green regions, climate variability is more weakly correlated with variability in crop yields. Based on a variety of variable feeding into their model, temperature and precipitation changes were less likely to explain changing crop yields.

Lighter green and yellow regions indicate that climate variability is more strongly correlated with crop yield variability. In these areas, changes in precipitation and temperature were very responsible for changes in output.

Next, we'll perform the same raster clipping operation as earlier using a vector file on Brazil's boundaries to narrow the scope of this analysis for maize production, analyzing whether crop variability as explained by climate variability differs in wetter and drier areas.

In [ ]:
pip install geopandas rioxarray rasterio

###Maize

```
import geopandas as gpd
import rioxarray as rxr
import xarray as xr

# Load the Brazil boundary GeoJSON
brazil = gpd.read_file("https://github.com/jlschwartz3/bio108tutorial/blob/main/brazil_Brazil_Country_Boundary.geojson")

# Load the raster and make sure it has spatial info
raster = rxr.open_rasterio(
    "/content/drive/MyDrive/crop yields/explanatorycat_maize.tif",
    masked=True
)

# Ensure the CRS matches (if not, reproject Brazil boundary)
if raster.rio.crs != brazil.crs:
    brazil = brazil.to_crs(raster.rio.crs)

# Clip the raster to Brazil
raster_clipped = raster.rio.clip(brazil.geometry, brazil.crs, drop=True)

# Optional: Save clipped raster if you want
raster_clipped.rio.to_raster("/content/drive/MyDrive/crop yields/maize_clipped_brazil.tif")
```

To plot the Leafmap using the Brazil vector boundaries, we can run the earlier leafmap code, but with the newly clipped data.

```
m = leafmap.Map(center=(-10, -52), zoom=4)  # Centered on Brazil

m.add_raster(
    "/content/drive/MyDrive/crop yields/maize_clipped_brazil.tif",
    layer_name="Brazil Maize",
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    nodata=0,
)

m.add_legend(title="Limiting Factors for Maize Growth", legend_dict=legend_dict)
m
```

In [ ]:
import rasterio

out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)

# Use the original profile and update it for the new clipped raster
out_meta = src.meta.copy()

out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save the clipped precip raster
with rasterio.open("/content/drive/MyDrive/crop yields/precip_brazil.tif", "w", **out_meta) as dest:
    dest.write(out_image)

    import rioxarray as rxr

precip = rxr.open_rasterio("/content/drive/MyDrive/crop yields/precip_brazil.tif", masked=True).squeeze()

# Load both rasters
yield_var = rxr.open_rasterio("/content/drive/MyDrive/crop yields/maize_clipped_brazil.tif", masked=True).squeeze()
precip = rxr.open_rasterio("/content/drive/MyDrive/crop yields/precip_brazil.tif", masked=True).squeeze()

# Reproject precipitation to match yield raster
precip = precip.rio.reproject_match(yield_var)

Next, we'll calculate the precipitation values from within the clipped raster.

```
import numpy as np
import xarray as xr

# Mask where there's no data in either
valid_mask = (~yield_var.isnull()) & (~precip.isnull())

# Extract valid values
yield_vals = yield_var.where(valid_mask).values.flatten()
precip_vals = precip.where(valid_mask).values.flatten()
```

Now, we'll find the average precipitation for each of the 7 categories identified by Ray et al. representing influence of climate variability.

```
import pandas as pd

df = pd.DataFrame({
    "yield_var_cat": yield_vals.astype(int),
    "precip": precip_vals
})

# Keep only yield_var_cat values between 1 and 7, excluding 0/No Data areas because of difficulty wrangling
df = df[df["yield_var_cat"].between(1, 7)]

# Remove NaNs (in case precip still has them)
df = df.dropna()

# Group by yield variability class
avg_precip_by_class = df.groupby("yield_var_cat")["precip"].mean()
print(avg_precip_by_class)
```

Finally, we'll make a simple bar graph to represent this data.

```
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 5))  # adjust size as needed

avg_precip_by_class.plot.bar(
    ax=ax,
    title="Avg Precip by Maize Yield Variability Explained by Climate Variability"
)

ax.set_ylabel("Average Monthly Precipitation (mm)")  # Add y-axis label
ax.set_xlabel("Yield Variability Class")      # Add x-axis label
plt.show()
```

#### **Questions**

1. What do we observe in this final graph? Does there seem to be a pattern?
2. In areas where climate (and precip) variability has more of an impact, how can farmers protect themselves against climate change? How can governments help?
3. Taking a step back, if farmers in certain regions are experiencing a lot of variability in crop yields, how might they adjust their decisions on sowing and harvesting? Could it drive them to abandon farming altogether?
